# Progress Task 1: Prediction of wine quality

# Introduction

This progress task has the aim to predict the quality of wine based on its physicochemical properties. The dataset used in this task is the [Wine Quality Dataset](https://archive.ics.uci.edu/dataset/186/wine+quality) from the UCI Machine Learning Repository. Credits to *P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.*

The objective of this task is to select an apropiate regression and classification model and compare them.

## Prepare environmental variables

Download the dataset and import the necessary packages.

In [ ]:
%pip install ucimlrepo seaborn matplotlib

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from ucimlrepo import fetch_ucirepo 
 
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
 
# data (as pandas dataframes) 
X = wine_quality.data.features 
y = wine_quality.data.targets 

df_wine = pd.concat([X,y], axis=1)
 
# metadata 
print(wine_quality.metadata) 

# get variable information 
print(wine_quality.variables)

## Exploratory Data Analysis (EDA)

In this section, a brief exploratory data analysis (EDA) will be performed on the dataset, prior to correctly pre-process it and capture the most relevant features for model training.

### Describing the dataset

In [ ]:
# Check the number of instances and the number of features
print ("Shape of data:", X.shape , y.shape)

In [ ]:
# Print the first rows of the features
print("=================== Feature's First Rows ===================\n", X.head(3), "\n")

# Print the first rows of the target
print("=================== Target's First Rows ===================\n", y.head(3), "\n")

In [ ]:
# Check for missing values
print("=================== Null value count ===================\n",df_wine.isnull().sum(), "\n")

In [ ]:
df_wine.describe(percentiles=[])

Now that we have taken a look into the dataset, here's a summary:

The dataset consists of 11 continuous features, none of them with missing values. The features are: 

* `fixed_acidity`: with values ranging from 4.6 to 15.9.
* `volatile_acidity`: with values ranging from 0.12 to 1.58.
* `citric_acid`: with values ranging from 0 to 1.66.
* `residual_sugar`: with values ranging from 0.6 to 65.8.
* `chlorides`: with values ranging from 0.009 to 0.611.
* `free_sulfur_dioxide`: with values ranging from 1 to 289.
* `total_sulfur_dioxide`: with values ranging from 6 to 440.
* `density`: with values ranging from 0.99 to 1.003.
* `pH`: with values ranging from 2.74 to 4.01.
* `sulphates`: with values ranging from 0.33 to 2.
* `alcohol`: with values ranging from 8.4 to 14.9.

The target variable is:
* `quality`: is an integer variable, from 0 to 10 but in this dataset it ranges from 3 to 9.

Now that the statistical summary of the dataset has been obtained, a pairplot will be created to visualize the relationships between the features and the target variable.

### Descriptive statistics

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='quality', data=df_wine)
plt.title("Distribution of Wine Quality Ratings")
plt.xlabel("Quality")
plt.ylabel("Count")
plt.show()

As shown in the plot, the target variable `quality` is not a balanced set. The majority of the wines have a quality of 5 or 6, with a few wines having a quality of 3 or 9. 

In [ ]:
sns.pairplot(df_wine, y_vars='quality',x_vars=df_wine.columns[:-1])
plt.show()

From the pairplot, a strange data distribution can be observed. All the instances seem to be grouped by a certain value of the variable `quality`. The reason for this is that the target variable is **discrete**, so **it is treated as a categorical variable**.

Given that no direct relation with the target can be inferred from the pairplot, the next step is to create pairplots between every pair of features. Then, the dependencies between the features will be analyzed.

In [ ]:
sns.pairplot(data = X)
plt.show()

From the pairplot there are some interesting observations: 

- Fixed acidity and density seem to have a linear relationship. 
- Density seems to have a horizontal line pattern with other features, that could represent a constant value.

From there, valueable information cannot be extracted, so it is necessary to continue analyzing the dataset. 

In [ ]:
# Correlation matrix
corrmat = X.corr()
sns.heatmap(corrmat, square = True, annot=True)

Out from the plot, the strongest correlation can be observed between the attributes **`free_sulfur_dioxide`** and **`total_sulfur_dioxide`** (0.72). The reason for this is total sulfur dioxide includes the free sulfur dioxide, so the variable free sulfur will be removed from the dataset, as both variables represent almost the same information and this will reduce redundancy.

The second strongest correlation is between **`density`** and **`alcohol`** (-0.69). This correlation is negative, due to the fact that an increase in the alcohol graduation in wine leads to a loss of water quantity. Therefore, given that alcohol is less dense than water, the density of the wine decreases.

maybe test to remove density as it might be a constant value?

In [ ]:
df_wine.drop(columns=['free_sulfur_dioxide'], inplace=True)

## Performance evaluation of regression models

We will put here the different model evaluations

### Random Forest Regressor

The Random Forest Regressor model is based on decorrelated trees that reduce the variance of the model and reduce overfitting. This is a powerful tool as our dataset is higly imbalance.

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import randint

We will use the parameter `stratify` in the `train_test_split` function to ensure that the distribution of the target variable is the same in the training and testing sets.

In [ ]:
X = df_wine.drop(columns=['quality'])
X_train, X_test, y_train, y_test = train_test_split(X, df_wine['quality'], test_size=0.2, random_state=42, stratify=df_wine['quality'])

In [ ]:
rf = RandomForestRegressor(random_state=42)

We have decided to use the `RandomizedSearchCV` to find the best hyperparameters as the performance is more or less equal to the `GridSearchCV` but it is 10 times faster (in this case). The hyperparameters that will be tuned are:
* n_estimators: the number of trees in the forest.
* max_depth: the maximum depth of the tree.
* min_samples_split: the minimum number of samples required to split an internal node.
* min_samples_leaf: the minimum number of samples required to be at a leaf node.

In [ ]:
# Define the parameter distribution for Random Search
param_dist = {
    'n_estimators': randint(50, 300),
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 5)
}
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                n_iter=50, cv=5, n_jobs=-1, verbose=2, random_state=42)

random_search.fit(X_train, y_train)
y_pred_random = random_search.predict(X_test)

# Print statics
print("Best Parameters from Random Search:", random_search.best_params_)
print("Best Score from Random Search:", random_search.best_score_)
print("----Test set----")
print("MSE = ", mean_squared_error(y_test, y_pred_random))
print("R2 = ", r2_score(y_test, y_pred_random))
print("MAE = ", mean_absolute_error(y_test, y_pred_random))


After training the model, the resulting best hyperparameters are:
* n_estimators: 138
* min_samples_split: 2
* min_samples_leaf: 1
* max_depth: 20

About the **performance metrics**, the best model achieves the following results:
* MSE = 0.387 which is lower than the variance of the target variable, so the mode's error is lower than the dispersion of the target variable.
* $R^2$ = 0.492 means that the model explains almost half of the variance of the `quality` variable.
* MAE = 0.441 means the error between the predicted and the true value is 0.441

Most of the `quality` values are between 7 and 5, so the model is better at predicting these values. 

In [ ]:
print("Variance of the variable quality = ", df_wine['quality'].var())

In [ ]:
plot_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_random}, index=X_test.index)
plot_df.sort_index(inplace=True)

# Plotting the actual vs predicted values
plt.figure(figsize=(16, 8))
plt.plot(plot_df.index, plot_df['Actual'], label='Actual', color='lightblue', marker='o', markersize=4)
plt.plot(plot_df.index, plot_df['Predicted'], label='Predicted', color='salmon', marker='x', markersize=4)

plt.title('Actual vs Predicted Values for Random Forest Regressor')
plt.ylabel('Quality')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

Now, let's check the importance of the features in the model to test if reducing the dimensionality of the dataset could improve the model.
This are the importance of the features in the random forest regressor model:

In [ ]:
print ('Feature Relevances')
print(pd.DataFrame({'Attributes': X_train.columns,
            'Feature importance':random_search.best_estimator_.feature_importances_}).sort_values('Feature importance', ascending=False))

The most important features is the `alcohol` whereas the least important is the `citric_acid`. Although training the model again with a subset of the features improves the time, it does not improve the performance of the model. Taking int account that the error should decrease and the $R^2$ should increase.

In [ ]:
#Uncomment for retraining with a subset
# new_df = df_wine[['alcohol', 'volatile_acidity', 'residual_sugar', 'total_sulfur_dioxide', 'sulphates', 'pH']]
# X_train, X_test, y_train, y_test = train_test_split(new_df, df_wine['quality'], test_size=0.2, random_state=42, stratify=df_wine['quality'])

# rf = RandomForestRegressor(random_state=42)
# param_dist = {
#     'n_estimators': randint(50, 300),
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': randint(2, 11),
#     'min_samples_leaf': randint(1, 5)
# }
# random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
#                                 n_iter=50, cv=5, n_jobs=-1, verbose=2, random_state=42)

# random_search.fit(X_train, y_train)
# y_pred_random_subset = random_search.predict(X_test)

# # Print statics
# print("Best Parameters from Random Search:", random_search.best_params_)
# print("Best Score from Random Search:", random_search.best_score_)
# print("----Test set----")
# print(f"MSE(subset) = {mean_squared_error(y_test, y_pred_random_subset):.3f} vs MSE(full) = {mean_squared_error(y_test, y_pred_random):.3f}")
# print(f"R2(subset) = {r2_score(y_test, y_pred_random_subset):.3f} vs R2(full) = {r2_score(y_test, y_pred_random):.3f}")
# print(f"MAE(subset) = {mean_absolute_error(y_test, y_pred_random_subset):.3f} vs MAE(full) = {mean_absolute_error(y_test, y_pred_random):.3f}")

**Conclusion for Random Forest Regressor**: This might be a good model if we want to focus on the most common values of the quality (5, 6 and 7). However, the model is not able to predict the extreme values of the quality (3 and 9).


### Final decision

Here we mention the best model.

## Performance evaluation of classification models

We will put here the different model evaluations

### Final decision

Here we mention the best model.